In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import syllables
import re
import xlsxwriter

<h1>Assignment 1</h1>
<h3>Data Extraction</h3>

In [ ]:
#extracting url

exel=pd.read_excel('C:\\Users\\Sambh\\Downloads\\Input.xlsx')
url=exel.loc[:,"URL"]
url

0      https://insights.blackcoffer.com/ai-in-healthc...
1      https://insights.blackcoffer.com/what-if-the-c...
2      https://insights.blackcoffer.com/what-jobs-wil...
3      https://insights.blackcoffer.com/will-machine-...
4      https://insights.blackcoffer.com/will-ai-repla...
                             ...                        
108    https://insights.blackcoffer.com/blockchain-fo...
109    https://insights.blackcoffer.com/the-future-of...
110    https://insights.blackcoffer.com/big-data-anal...
111    https://insights.blackcoffer.com/business-anal...
112    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 113, dtype: object

In [ ]:
#writting title and text in URL_ID.txt file

for i in range(len(url)):
    ind_url=url[i]
    try:
        page=requests.get(ind_url)
        soup=BeautifulSoup(page.content,'html.parser')

        #Extracting Heading
        title=soup.find_all(attrs={'class':'td-post-title'})
        heading0=title[0].text.split("\n")
        f = open("C:\\Users\\Sambh\\Desktop\\url_id.txt", "a")
        f.write(heading0[1]+'\n\n')
        f.close()

        #Extracting Body
        body=soup.find_all(attrs={"class","td-post-content tagdiv-type"})
        bdy=body[0].text.replace("\n", "")
        f = open("C:\\Users\\Sambh\\Desktop\\url_id.txt", "a")
        f.write(bdy+'\n\n\n\n\n\n')
        f.close()
    except:
        continue

<h1>Assignment 2</h1>
<h3>Sentiment Analysis</h3>

In [ ]:
#extracting url

df=pd.read_excel("C:\\Users\\Sambh\\Downloads\\Output Data Structure.xlsx")
out_url=df.loc[:,"URL"]
out_url

0      https://insights.blackcoffer.com/ai-in-healthc...
1      https://insights.blackcoffer.com/what-if-the-c...
2      https://insights.blackcoffer.com/what-jobs-wil...
3      https://insights.blackcoffer.com/will-machine-...
4      https://insights.blackcoffer.com/will-ai-repla...
                             ...                        
109    https://insights.blackcoffer.com/blockchain-fo...
110    https://insights.blackcoffer.com/the-future-of...
111    https://insights.blackcoffer.com/big-data-anal...
112    https://insights.blackcoffer.com/business-anal...
113    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 114, dtype: object

In [ ]:
#creating a combination dictionary of stop words

f1 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_Auditor.txt','r')
f1_r=f1.read()
sw_auditor=f1_r.replace("\n", " ").split(" ")

f2 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_Currencies.txt','r')
f2_r=f2.read()
sw_cur=f2_r.replace("\n", " ").split(" ")

f3 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_DatesandNumbers.txt','r')
f3_r=f3.read()
sw_DN=f3_r.replace("\n", " ").split(" ")

f4 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_Generic.txt','r')
f4_r=f4.read()
sw_gen=f4_r.replace("\n", " ").split(" ")

f5 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_GenericLong.txt','r')
f5_r=f5.read()
sw_genlong=f5_r.replace("\n", " ").split(" ")

f6 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_Geographic.txt','r')
f6_r=f6.read()
sw_geo=f6_r.replace("\n", " ").split(" ")

f7 =open('C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\StopWords\\StopWords_Names.txt','r')
f7_r=f7.read()
sw_names=f7_r.replace("\n", " ").split(" ")

rmv_wrd=sw_auditor + sw_cur + sw_DN + sw_gen + sw_genlong +sw_geo +sw_names


#reading positive and negative dictionaries

pf=open("C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\MasterDictionary\\positive-words.txt")
pf_r=pf.read()
pos_wrd=pf_r.replace("\n"," ").split(" ")

nf=open("C:\\Users\\Sambh\\Downloads\\StopWords-20230407T141210Z-001\\MasterDictionary\\negative-words.txt")
nf_r=nf.read()
neg_wrd=nf_r.replace("\n"," ").split(" ")

In [ ]:
workbook = xlsxwriter.Workbook("C:\\Users\\Sambh\\Downloads\\Output Data Structure(1).xlsx")
worksheet = workbook.add_worksheet()
for i in range(0,114):
    try:
        info=requests.get(out_url[i])
        content=BeautifulSoup(info.content,'html.parser')
        text=content.find_all(attrs={"class","td-post-content tagdiv-type"})
        Clean_text=text[0].text.replace("\n", "").split(" ")
        cln_txt=text[0].text.replace("\n", "")
        no_sentence=text[0].text.replace("\n", "").split(".")

        #removing stop words
        resultwords  = [word for word in Clean_text if word.lower() not in rmv_wrd]
        result = ' '.join(resultwords)

        cmplx=0
        syll_count=0
        wrd_count=0
        for j in range (0,len(Clean_text)):
            syll=syllables.estimate(Clean_text[j])
            if(syll>2):
                cmplx=cmplx+1
            syll_count=syll_count+1
            wrd_count=wrd_count+len(Clean_text[j])

        pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
        pronouns = pronounRegex.findall(cln_txt)
        per_pro=len(pronouns)

        lst3 = [value for value in pos_wrd if value in result]
        pos_score=len(lst3)

        lst4 = [value for value in neg_wrd if value in result]
        neg_score=len(lst4)

        Polarity_Score = (pos_score-neg_score )/ ((pos_score + neg_score) + 0.000001)

        Subjectivity_Score = (pos_score + neg_score)/ ((len(result)) + 0.000001)

        Avg_Sentence_Length = len(Clean_text) / len(no_sentence)

        Percentage_of_Complex_words = (cmplx / len(Clean_text))*100

        Fog_Index = 0.4 * (Avg_Sentence_Length + Percentage_of_Complex_words)

        Average_Number_of_Words = len(Clean_text) / len(no_sentence)

        syllable_count=(syll_count/wrd_count)*len(Clean_text)

        Avg_word_len=wrd_count/len(Clean_text)

        word=len(resultwords)

        #writting on exel output

        cC='C'+str(i+1)
        cD='D'+str(i+1)
        cE='E'+str(i+1)
        cF='F'+str(i+1)
        cG='G'+str(i+1)
        cH='H'+str(i+1)
        cI='I'+str(i+1)
        cJ='J'+str(i+1)
        cK='K'+str(i+1)
        cL='L'+str(i+1)
        cM='M'+str(i+1)
        cN='N'+str(i+1)
        cO='O'+str(i+1)

        worksheet.write(cC, pos_score)
        worksheet.write(cD, neg_score)
        worksheet.write(cE, Polarity_Score)
        worksheet.write(cF, Subjectivity_Score)
        worksheet.write(cG, Avg_Sentence_Length)
        worksheet.write(cH, Percentage_of_Complex_words)
        worksheet.write(cI, Fog_Index)
        worksheet.write(cJ, Average_Number_of_Words)
        worksheet.write(cK, cmplx)
        worksheet.write(cL, word)
        worksheet.write(cM, syllable_count)
        worksheet.write(cN, per_pro)
        worksheet.write(cO, Avg_word_len)
    except:
        continue
        print("Error")
workbook.close()

<h1>Created By~</h1>
<h2>Sambhav Srivastava</h2>